In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
import geopandas as gpd
from scipy.spatial import cKDTree
import folium
from folium.plugins import MarkerCluster
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
collisions = pd.read_csv("/content/drive/My Drive/MIE368 Project - Group 15/Code/collisions_dataset/collisions_dataset_new/collisions_enriched.csv")
speed_cameras = pd.read_csv("/content/drive/My Drive/MIE368 Project - Group 15/Code/speed_camera_dataset/speed_camera_clean_new/speed_cameras_clean.csv")

In [5]:
print(collisions.head())

         date  hour        dow        lat        lon              severity  \
0  2014-01-01    17  Wednesday  43.701225 -79.377616  Property Damage Only   
1  2014-01-01    14  Wednesday  43.726091 -79.397589  Property Damage Only   
2  2014-01-01     4  Wednesday  43.762676 -79.336644  Property Damage Only   
3  2014-01-01    11  Wednesday  43.703234 -79.346615  Property Damage Only   
4  2014-01-01     1  Wednesday  43.650410 -79.378428  Property Damage Only   

   wx_precip_day  wx_precipitation  wx_rain  wx_snow  ...  \
0              0               0.0      0.0      0.0  ...   
1              0               0.0      0.0      0.0  ...   
2              0               0.0      0.0      0.0  ...   
3              0               0.0      0.0      0.0  ...   
4              0               0.0      0.0      0.0  ...   

   wx_avg_hourly_wind_speed  wx_max_relative_humidity  \
0                     19.38                      75.0   
1                     19.38                      7

In [6]:
print(speed_cameras.head())

         lon        lat status_clean  ward_num  \
0 -79.567451  43.713609       active         1   
1 -79.550510  43.700973       active         1   
2 -79.561386  43.728553       active         1   
3 -79.597523  43.748744       active         1   
4 -79.553404  43.722559       active         1   

                                    location  fid  
0        Kipling Ave. North of Rexdale Blvd.    1  
1   St. Andrews Blvd. West of Islington Ave.    2  
2     Islington Ave. North of Fordwich Cres.    3  
3  Martin Grove Rd. South of Silverstone Dr.    4  
4           Golfdown Dr. East of Turpin Ave.    5  


In [7]:
collisions = collisions.dropna(subset=['lat', 'lon'])
speed_cameras = speed_cameras.dropna(subset=['lat', 'lon'])

In [8]:
# convert to GeoDataFrames
coll_gdf = gpd.GeoDataFrame(collisions, geometry=gpd.points_from_xy(collisions['lon'], collisions['lat']), crs="EPSG:4326")
cam_gdf  = gpd.GeoDataFrame(speed_cameras, geometry=gpd.points_from_xy(speed_cameras['lon'], speed_cameras['lat']), crs="EPSG:4326")

# convert to UTM (meters)
coll_utm = coll_gdf.to_crs(epsg=32617)
cam_utm = cam_gdf.to_crs(epsg=32617)

In [9]:
# build KD-Tree for nearest-neighbour distance
camera_coords = np.array(list(zip(cam_utm.geometry.x, cam_utm.geometry.y)))
collision_coords = np.array(list(zip(coll_utm.geometry.x, coll_utm.geometry.y)))

tree = cKDTree(camera_coords)
distances, _ = tree.query(collision_coords, k=1)
coll_utm['distance_to_camera_m'] = distances

In [10]:
# 500m buffer coverage
buffer_radius = 500
cam_utm['buffer'] = cam_utm.geometry.buffer(buffer_radius)
all_buffers = cam_utm['buffer'].unary_union

coll_utm['within_500m'] = coll_utm.geometry.within(all_buffers)

/tmp/ipython-input-2816692917.py:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  all_buffers = cam_utm['buffer'].unary_union


In [11]:
coverage_rate = coll_utm['within_500m'].mean()
avg_distance = coll_utm['distance_to_camera_m'].mean()

print("\nBaseline Coverage:")
print(f"Collision Coverage Within 500m: {coverage_rate:.3f}")
print(f"Average Distance to Nearest Camera (m): {avg_distance:.1f}")

# add Near/Far field for statistical tests
coll_utm['near_far'] = np.where(coll_utm['within_500m'], 'Near', 'Far')


Baseline Coverage:
Collision Coverage Within 500m: 0.270
Average Distance to Nearest Camera (m): 821.9


In [12]:
print(collisions.columns.tolist())

['date', 'hour', 'dow', 'lat', 'lon', 'severity', 'wx_precip_day', 'wx_precipitation', 'wx_rain', 'wx_snow', 'wx_snow_on_ground', 'wx_max_temperature', 'wx_avg_temperature', 'wx_avg_hourly_temperature', 'wx_min_temperature', 'wx_max_humidex', 'wx_min_windchill', 'wx_max_wind_speed', 'wx_avg_hourly_wind_speed', 'wx_max_relative_humidity', 'wx_avg_relative_humidity', 'wx_precip_amount_any', 'cam_nearest_m', 'cam_within_250m', 'cam_location', 'cam_status_clean', 'cam_ward_num', 'cam_fid']


In [13]:
# weather-collision correlation
daily_data = (
    collisions.groupby('date')
    .agg({'severity':'count','wx_precip_amount_any':'mean','wx_snow':'mean'})
    .rename(columns={'severity':'collision_count'})
    .dropna()
    .reset_index()
)

corr_precip, p_precip = stats.pearsonr(daily_data['wx_precip_amount_any'], daily_data['collision_count'])
corr_snow, p_snow = stats.pearsonr(daily_data['wx_snow'], daily_data['collision_count'])

print("\nWeather Coverage:")
print(f"Precipitation vs Collisions: r={corr_precip:.3f}, p={p_precip}")
print(f"Snow vs Collisions:          r={corr_snow:.3f}, p={p_snow}")


Weather Coverage:
Precipitation vs Collisions: r=0.123, p=6.95406735394006e-16
Snow vs Collisions:          r=0.217, p=8.74439825817612e-47


In [14]:
# severity encoding
coll_utm['severity_num'] = coll_utm['severity'].map({
    'Property Damage Only': 1,
    'Injury': 2,
    'Non-Fatal Injury': 2,
    'Fatal': 3
})

# time of day for biases
coll_utm['day_night'] = coll_utm['hour'].apply(lambda h: 'Day' if 7 <= h <= 18 else 'Night')

In [15]:
# Statistical Tests
# T-test: Severity near vs far
near = coll_utm[coll_utm['near_far']=='Near']['severity_num']
far = coll_utm[coll_utm['near_far']=='Far']['severity_num']

sample_n = min(len(near), len(far), 10000)
near_s = near.sample(sample_n, random_state=42)
far_s = far.sample(sample_n, random_state=42)

t,p = stats.ttest_ind(near_s, far_s, equal_var=False)

print("T-Test: Near vs Far")
print(f"t={t:.4f}, p={p:.6f}")

# Chi-square tests
print("\nCh-Squared Test")

chi1 = stats.chi2_contingency(pd.crosstab(coll_utm['severity'], coll_utm['near_far']))
chi2 = stats.chi2_contingency(pd.crosstab(coll_utm['day_night'], coll_utm['severity']))

print(f"Severity vs Camera Presence: p={chi1[1]}")
print(f"Lighting vs Severity:        p={chi2[1]}")

# ANOVA across months
coll_utm['month'] = pd.to_datetime(coll_utm['date'], errors='coerce').dt.month
anova_groups = [g['severity_num'].sample(min(len(g), 10000), random_state=42)
                for _,g in coll_utm.groupby('month') if g['severity_num'].nunique()>1]

f,p = stats.f_oneway(*anova_groups)
print("\nANOVA: Monthly Severity")
print(f"F={f:.4f}, p={p}")

T-Test: Near vs Far
t=1.8626, p=0.062531

Ch-Squared Test
Severity vs Camera Presence: p=4.8566599920991305e-05
Lighting vs Severity:        p=1.351574303095472e-122

ANOVA: Monthly Severity
F=13.5216, p=2.811058501711808e-26


In [16]:
# K-Means clustering
coords = collisions[['lon','lat']].dropna()
kmeans = KMeans(n_clusters=4, n_init=10, random_state=42)
collisions['k_cluster'] = kmeans.fit_predict(coords)

print("\nK-Means Cluster:")
print(collisions['k_cluster'].value_counts(normalize=True))


K-Means Cluster:
k_cluster
1    0.321553
0    0.302062
3    0.216626
2    0.159759
Name: proportion, dtype: float64


In [17]:
# DBSCAN Hotspot Clustering
coords_utm = np.array(list(zip(coll_utm.geometry.x, coll_utm.geometry.y)))
db = DBSCAN(eps=300, min_samples=5).fit(coords_utm)
coll_utm['db_cluster'] = db.labels_

print("\nDBSCAN Hotspots:")
print(coll_utm['db_cluster'].value_counts())

# hotspots outside camera coverage
hotspots = coll_utm[coll_utm['db_cluster'] != -1]
hotspots_outside = hotspots[~hotspots['within_500m']]
print(f"Hotspots outside camera zones: {len(hotspots_outside)}")


DBSCAN Hotspots:
db_cluster
0      553492
5       18116
21       5421
10       3943
31       2925
        ...  
190         8
126         7
196         7
197         6
188         5
Name: count, Length: 200, dtype: int64
Hotspots outside camera zones: 472014


In [18]:
# calculate tot # of collision points and number of points covered by cameras
total_collisions = len(coll_utm)
covered_collisions = coll_utm['within_500m'].sum()

# calculate the coverage percentage
coverage_percent_baseline = (covered_collisions / total_collisions) * 100
print(f"Coverage Percentage: {coverage_percent_baseline:.2f}%")

Coverage Percentage: 26.98%


In [21]:
# Baseline summary table

summary = pd.DataFrame({
    "Metric": [
        "Coverage rate (500m)",
        "Avg distance to nearest camera (m)",
        "Precipitation correlation",
        "Snow correlation",
        "T-test p-value (Near vs Far severity)",
        "Chi-square p: severity vs camera",
        "Chi-square p: lighting vs severity",
        "ANOVA p-value (seasonal severity)",
        "DBSCAN hotspots",
        "Hotspots outside coverage",
        "Baseline Coverage Percentage"
    ],
    "Value": [
        coverage_rate,
        avg_distance,
        corr_precip,
        corr_snow,
        p,
        chi1[1],
        chi2[1],
        p,
        len(hotspots),
        len(hotspots_outside),
        coverage_percent_baseline
    ]
})


print("\n\nBaseline Summary:")
print(summary)



Baseline Summary:
                                   Metric          Value
0                    Coverage rate (500m)   2.698195e-01
1      Avg distance to nearest camera (m)   8.218678e+02
2               Precipitation correlation   1.227848e-01
3                        Snow correlation   2.167259e-01
4   T-test p-value (Near vs Far severity)   2.811059e-26
5        Chi-square p: severity vs camera   4.856660e-05
6      Chi-square p: lighting vs severity  1.351574e-122
7       ANOVA p-value (seasonal severity)   2.811059e-26
8                         DBSCAN hotspots   6.464380e+05
9               Hotspots outside coverage   4.720140e+05
10           Baseline Coverage Percentage   2.698195e+01
